In [1]:
from scipy.stats import norm
import numpy as np
import matplotlib.pyplot as plt
from photutils.aperture import aperture_photometry, CircularAperture, CircularAnnulus
from astropy.stats import sigma_clipped_stats

from pathlib import Path
from astropy.nddata import CCDData
from astropy.io import fits
from ccdproc import ImageFileCollection
import ccdproc as ccdp

from astropy.nddata import CCDData
from astropy.stats import mad_std
import astropy.units as u
from convenience_functions import show_image
from astropy.timeseries import BinnedTimeSeries, TimeSeries
from astropy.table import Column, Table

from astropy import units as u
from astropy.coordinates import SkyCoord
from photutils.aperture import SkyCircularAperture

from astropy.modeling.fitting import LevMarLSQFitter
from astropy.stats import gaussian_sigma_to_fwhm
from photutils.background import MADStdBackgroundRMS, MMMBackground
from photutils.detection import IRAFStarFinder
from photutils.psf import (DAOGroup, IntegratedGaussianPRF,
                           IterativelySubtractedPSFPhotometry)

# suppress warnings is not recommended :I
import warnings
warnings.filterwarnings("ignore")

In [5]:
cc_com_reduced = './data/cc_com/reduced/'
tz_boo_reduced = './data/tz_boo/reduced/'

def get_aperture_fluxes(images_dir, image_dim=(3056, 3056), r=50.0):
    images = ImageFileCollection(images_dir)
    images = images.files_filtered(include_path=True)
    
    APT_fluxes = []
    APT_unc = []
    ANU_fluxes = []
    for i, path in np.ndenumerate(images):
        # print(path)
        image = CCDData.read(path)
        mean, median, stdev = sigma_clipped_stats(image.data, sigma=3.0)
        image.data = image.data - median
        position = np.transpose((
            image.header['X-CENTROID'],
            image.header['Y-CENTROID']
        ))
        aperture = CircularAperture(position, r=r)
        annul = CircularAnnulus(position, r_in=r, r_out=1.5*r)
        phot_circ = aperture_photometry(image, aperture)[0]['aperture_sum']
        phot_anul = aperture_photometry(image, annul)[0]['aperture_sum']
        
        image.header['APT-FLUX'] = phot_circ.value
        image.header['ANU-FLUX'] = phot_anul.value
        
        APT_fluxes.append(phot_circ.value)
        ANU_fluxes.append(phot_anul.value)
        
    return APT_fluxes, ANU_fluxes

def get_psf_fluxes(images_dir, results_dir, image_dim=(3056, 3056), sigma_psf=2.0):
    images = ImageFileCollection(images_dir)
    images = images.files_filtered(include_path=True)
    
    info = BinnedTimeSeries.read(
        results_dir,
        time_bin_start_column='time_bin_start',
        time_bin_size_column='time_bin_size',
        time_bin_size_unit=u.s
    )
    
    bkgrms = MADStdBackgroundRMS()
    daogroup = DAOGroup(2.0 * sigma_psf * gaussian_sigma_to_fwhm)
    mmm_bkg = MMMBackground()
    fitter = LevMarLSQFitter()
    psf_model = IntegratedGaussianPRF(sigma=sigma_psf)
    
    psf_model.x_0.fixed = True
    psf_model.y_0.fixed = True
    
    flux_fit = []
    flux_unc = []
    for i, path in np.ndenumerate(images):
        image = CCDData.read(path)
        xcentroid = info[i[0]]['xcentroid']
        ycentroid = info[i[0]]['ycentroid']
        centroid = Table(
            names=['x_0', 'y_0'],
            data=[[xcentroid], [ycentroid]]
        )
        
        std = bkgrms(image)
        iraffind = IRAFStarFinder(
            threshold=3.5*std,
            fwhm=sigma_psf * gaussian_sigma_to_fwhm,
            minsep_fwhm=0.01, roundhi=5.0, roundlo=-5.0,
            sharplo=0.0, sharphi=2.0
        )
        photometry = IterativelySubtractedPSFPhotometry(
            finder=iraffind,
            group_maker=daogroup,
            bkg_estimator=mmm_bkg,
            psf_model=psf_model,
            fitter=LevMarLSQFitter(),
            niters=1, fitshape=(11, 11)
        )
        result_tab = photometry(image=image, init_guesses=centroid)
        flux_fit.append(result_tab[0]['flux_fit'])
        flux_unc.append(result_tab[0]['flux_unc'])
        
    return flux_fit, flux_unc
        
    
    
    
    

In [3]:
tzboo_apt, tzboo_anu = get_aperture_fluxes(tz_boo_reduced)
tzboo_psf, tzboo_psf_unc = get_psf_fluxes(
    tz_boo_reduced,
    './results/tzboo_DAOresults.csv'
)
tzboo_DAO = BinnedTimeSeries.read(
    './results/tzboo_DAOresults.csv',
    time_bin_start_column='time_bin_start',
    time_bin_size_column='time_bin_size',
    time_bin_size_unit=u.s
)
apflux_col = Column(name='AP flux', data=tzboo_apt)
anflux_col = Column(name='Anu flux', data=tzboo_anu)
psfflux_col = Column(name='PSF flux', data=tzboo_psf)
psfunc_col = Column(name='PSF unc', data=tzboo_psf_unc)
tzboo_DAO.add_column(apflux_col)
tzboo_DAO.add_column(anflux_col)
tzboo_DAO.add_column(psfflux_col)
tzboo_DAO.add_column(psfunc_col)
tzboo_DAO.write('./results/tzboo_results.csv', overwrite=True)
tzboo_DAO

time_bin_start,time_bin_size,xcentroid,ycentroid,DAO flux,AP flux,Anu flux,PSF flux,PSF unc
,s,,,,,,,
Time,float64,float64,float64,float64,float64,float64,float64,float64
2023-05-05T03:37:43.990,15.0,1505.658545068577,1451.9598233141166,45.283864574166145,173605.72127367108,10691.38698440312,137640.22829637493,1431.0294440186078
2023-05-05T04:06:02.330,15.0,1486.5070405122656,1459.4518074772132,70.99202021518356,186632.4027813351,5426.347139751565,166013.05944002734,2777.9820578538393
2023-05-05T04:33:47.270,15.0,1529.8081988179474,1527.4394505568735,128.77656611821413,194173.06511433184,3433.299039635063,187405.0811357585,5124.707798573422
2023-05-05T05:33:33.170,15.0,1524.2979148448526,1521.6222535867441,102.93023116168068,169364.01312370133,3295.460832058391,166884.86401793937,4311.150260023957


In [7]:
cccom_apt, cccom_anu = get_aperture_fluxes(cc_com_reduced)
cccom_psf, cccom_psf_unc = get_psf_fluxes(
    cc_com_reduced,
    './results/cccom_DAOresults.csv'
)
cccom_DAO = BinnedTimeSeries.read(
    './results/cccom_DAOresults.csv',
    time_bin_start_column='time_bin_start',
    time_bin_size_column='time_bin_size',
    time_bin_size_unit=u.s
)
apflux_col = Column(name='AP flux', data=cccom_apt)
anflux_col = Column(name='Anu flux', data=cccom_anu)
psfflux_col = Column(name='PSF flux', data=cccom_psf)
psfunc_col = Column(name='PSF unc', data=cccom_psf_unc)
cccom_DAO.add_column(apflux_col)
cccom_DAO.add_column(anflux_col)
cccom_DAO.add_column(psfflux_col)
cccom_DAO.add_column(psfunc_col)
cccom_DAO.write('./results/cccom_results.csv', overwrite=True)
cccom_DAO

time_bin_start,time_bin_size,xcentroid,ycentroid,DAO flux,AP flux,Anu flux,PSF flux,PSF unc
,s,,,,,,,
Time,float64,float64,float64,float64,float64,float64,float64,float64
2023-05-05T03:47:49.549,15.0,1553.040442438791,1437.5034251892966,3.319052059965849,11295.962546012051,-869.5947339270268,8119.758385985809,193.2683156291756
2023-05-05T04:15:01.570,15.0,1551.4548739944155,1438.76161766664,4.407717909508814,10757.877316247144,150.70729613382568,9615.949033060508,235.36463805701268
2023-05-05T04:58:12.580,15.0,1550.1356753563516,1438.672920101272,5.94613799804669,10414.331066902712,394.93399798479186,10446.973498143943,326.5663205394534
